# Setup a regression experiment

In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
import h5py
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

dir_plots="z_0_plots"
dir_shape_functions="z_0_shape_functions"

hf = h5py.File('get_subhalos/TNG100_1_z0_sh0.h5', 'r')
data = {"total_mass":       np.log10((hf["total_mass"].value*10**10/0.6774)),\
        "gas_metallicity":  np.log10((hf["gas_metallicity"].value/0.0127)),\
        "star_metallicity": np.log10((hf["star_metallicity"].value/0.0127)),\
        "gas_mass":         np.log10(hf["masses"].value.T[0]*10**10/0.6774),\
        "dm_mass":          np.log10(hf["masses"].value.T[1]*10**10/0.6774),\
        "star_mass":        np.log10(hf["masses"].value.T[4]*10**10/0.6774),\
        "bh_mass":          np.log10(hf["masses"].value.T[5]*10**10/0.6774),\
        "bh_mass_dot":      np.log10(hf["bh_mass_dot"].value*(10**10/0.6774)/(0.978*10**9/0.6774)),\
        "sfr":              np.log10(hf["sfr"].value)}

df = pd.DataFrame(data=data)

df=df[(df["total_mass"]>0) &\
      (np.isfinite(df["dm_mass"])) &\
      (np.isfinite(df["star_mass"])) &\
      (np.isfinite(df["gas_mass"])) &\
      (np.isfinite(df["bh_mass"])) &\
      (np.isfinite(df["gas_metallicity"])) &\
      (np.isfinite(df["star_metallicity"])) &\
      (np.isfinite(df["bh_mass_dot"])) &\
      (np.isfinite(df["sfr"])) ]# &\
#       (np.log10(df["sfr"]/(10**df["star_mass"]))<-11)]
seed = 1

columns = ["bh_mass","bh_mass_dot","gas_mass","gas_metallicity","sfr","star_mass","star_metallicity"]

# columns = ["star_mass"]


x = df[columns]
y = df["dm_mass"]

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=seed)

In [2]:
len(X_train), len(X_test)

(10561, 4527)

# Explore the dataset

In [3]:
from interpret import show
from interpret.data import Marginal, ClassHistogram

marginal = Marginal(max_scatter_samples=len(X_train)).explain_data(X_train, y_train, name = 'Train Data Marginal')
show(marginal)

<!-- http://127.0.0.1:7513/140596567587024/ -->

# Train the Explainable Boosting Machine (EBM) and other Explainable Models (Linear Regression, Regression Tree)

In [ ]:
from interpret.glassbox import ExplainableBoostingRegressor, LinearRegression, RegressionTree

ebm = ExplainableBoostingRegressor(random_state=seed, interactions=3)
ebm.fit(X_train, y_train)

lr = LinearRegression(random_state=seed)
lr.fit(X_train, y_train)

rt = RegressionTree(random_state=seed)
rt.fit(X_train, y_train)

# Global Explanations: what the models learned overall

In [ ]:
ebm_global = ebm.explain_global(name='EBM')
lr_global  = lr.explain_global(name='Linear Regression')
rt_global  = rt.explain_global(name='Regression Tree')

# show(ebm_global)
# show(lr_global)
# show(rt_global)

# Local Explanations: how an individual prediction was made

In [ ]:
ebm_local = ebm.explain_local(X_test, y_test, name='EBM')
lr_local  = lr.explain_local(X_test, y_test, name='Linear Regression')
rt_local  = rt.explain_local(X_test, y_test, name='Regression Tree')

# show(ebm_local)
# show(lr_local)
# show(rt_local)

# Evaluate EBM, LR and RT perfomances

In [ ]:
from interpret.perf import RegressionPerf

ebm_perf = RegressionPerf(ebm.predict).explain_perf(X_test, y_test, name='EBM')
lr_perf = RegressionPerf(lr.predict).explain_perf(X_test, y_test, name='Linear Regression')
rt_perf = RegressionPerf(rt.predict).explain_perf(X_test, y_test, name='Regression Tree')

# show(ebm_perf)
# show(lr_perf)
# show(rt_perf)

# Dashboard: look at everything at once

In [ ]:
show([marginal, lr_global, lr_perf, lr_local, rt_global, rt_perf, rt_local, ebm_global, ebm_perf, ebm_local])